In [1]:
import os
import random
import numpy as np
import sys
import torch
import pprint

root = '/'
pprint.pprint(sys.path)

import_path = root + 'pigvae_all'
sys.path.append(import_path)
pprint.pprint(sys.path)

['/project/dsc-is/daiki-ko/Graph-Data_for_MCMC',
 '/opt/anaconda3/lib/python38.zip',
 '/opt/anaconda3/lib/python3.8',
 '/opt/anaconda3/lib/python3.8/lib-dynload',
 '',
 '/home/is/daiki-ko/.local/lib/python3.8/site-packages',
 '/opt/anaconda3/lib/python3.8/site-packages',
 '/opt/anaconda3/lib/python3.8/site-packages/locket-0.2.1-py3.8.egg',
 '/opt/anaconda3/lib/python3.8/site-packages/IPython/extensions',
 '/home/is/daiki-ko/.ipython']
['/project/dsc-is/daiki-ko/Graph-Data_for_MCMC',
 '/opt/anaconda3/lib/python38.zip',
 '/opt/anaconda3/lib/python3.8',
 '/opt/anaconda3/lib/python3.8/lib-dynload',
 '',
 '/home/is/daiki-ko/.local/lib/python3.8/site-packages',
 '/opt/anaconda3/lib/python3.8/site-packages',
 '/opt/anaconda3/lib/python3.8/site-packages/locket-0.2.1-py3.8.egg',
 '/opt/anaconda3/lib/python3.8/site-packages/IPython/extensions',
 '/home/is/daiki-ko/.ipython',
 '/project/dsc-is/daiki-ko/pigvae_all']


In [2]:
#from mol2graph.py
possible_atom_list = [
          'Si', 'C', 'N', 'S', 'P', 'O', 'F', 'Cl', 'Br', 'I', 'B'
      ]

max_num_nodes = 26

In [11]:
import numpy as np
import torch
from torch.utils.data import Dataset
from torch.utils.data.distributed import DistributedSampler
import random
import pytorch_lightning as pl
from torch_geometric.data import Data
from torch_geometric.utils import from_networkx
import networkx as nx
from networkx.algorithms.shortest_paths.dense import floyd_warshall_numpy

from networkx.generators.random_graphs import *
from networkx.generators.ego import ego_graph
from networkx.generators.geometric import random_geometric_graph

In [12]:
def smiles_to_graphs(smiles_list):
    
    node_features = []
    edge_features = []
    masks = []
    props = []
    
    for id, smi in enumerate(smiles_list):
        
        mol = Chem.MolFromSmiles(smi)
        
        mol_graph = mol2vec(mol)
        atoms = mol.GetAtoms()
        bonds = mol.GetBonds()

        atoms_list = np.arange(len(atoms))
        bonds_list = []

        for bond in bonds:

            begin_atom, end_atom = bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()
            bonds_list.append((begin_atom, end_atom))

        mg = nx.Graph()
        mg.add_nodes_from(atoms_list)
        mg.add_edges_from(bonds_list)

        num_edges = mg.number_of_edges()
        num_nodes = mg.number_of_nodes()
        num_nodes_init = mg.number_of_nodes()

        props.append(torch.Tensor([num_nodes]))
        mg.add_nodes_from([i for i in range(num_nodes, max_num_nodes)])
        nf = torch.zeros(max_num_nodes, num_node_f
        node_f = nf.unsqueeze(0)

        dm = torch.from_numpy(floyd_warshall_numpy(mg)).long()
        dm = torch.clamp(dm, 0, 5).unsqueeze(-1)
        num_nodes = dm.size(1)
        dm = torch.zeros((num_nodes, num_nodes, 6 + num_edge_f + 1)).type_as(dm).scatter_(2, dm, 1).float()
        dm[:,:, 6] = 1.0

        i_list = list(range(0, len(mol_graph.edge_index[0]), 2))
        j_list = list(range(1, len(mol_graph.edge_index[0]), 2))
        
        for idx, (i, j) in enumerate(zip(mol_graph.edge_index[0][i_list], mol_graph.edge_index[0][j_list])):
            dm[i,j,7:7+num_edge_f] = mol_graph.edge_attr[idx]
            dm[j,i,7:7+num_edge_f] = mol_graph.edge_attr[idx]
            dm[i,j,6] = 0.0
            dm[j,i,6] = 0.0

        for idx, node_x in enumerate(mol_graph.x):
            node_f[0][idx] = node_x

        edge_features.append(dm)
        masks.append((torch.arange(max_num_nodes) < num_nodes_init).unsqueeze(0))
        node_features.append(node_f)
        
    return node_features, edge_features, masks, props

In [19]:
task_names = ["bace", "mmp2", "malaria", "esol", "freesolv", "lipo", "vp", "bp", "logp"]

load_dir = root + "dataset/eval_dataset/molecular_properties/zinc/"

for task in task_names:
    
    data_dir = load_dir + "smiles/" + task  + "/"
    save_dir = load_dir + "graphs/" + task  + "/"
    
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    
    smiles_list = np.load(data_dir + "all_smiles.npy")
    target_list = np.load(data_dir + "all_ys.npy")
    
    node_features, edge_features, masks, props = smiles_to_graphs(smiles_list)
    all_targets = torch.from_numpy(np.array(target_list).astype(np.float32)).clone()
    
    node_features = torch.cat(node_features, dim=0)
    edge_features = torch.stack(edge_features, dim=0)
    masks = torch.cat(masks, dim=0)
    props = torch.cat(props, dim=0)
    
    with open(save_dir + "node_features.pickle", 'wb') as fa:
        pickle.dump(node_features, fa)
    fa.close
          
    with open(save_dir + "edge_features.pickle", 'wb') as fb:
        pickle.dump(edge_features, fb)
    fb.close
          
    with open(save_dir + "masks.pickle", 'wb') as fc:
        pickle.dump(masks, fc)
    fc.close

    with open(save_dir + "props.pickle", 'wb') as fd:
        pickle.dump(props, fd)
    fd.close

    with open(save_dir + "targets.pickle", 'wb') as fe:
        pickle.dump(all_targets, fe)
    fe.close
    
    print(save_dir)

molecular_properties/zinc/graphs/bace/
molecular_properties/zinc/graphs/mmp2/
molecular_properties/zinc/graphs/malaria/
molecular_properties/zinc/graphs/esol/
molecular_properties/zinc/graphs/freesolv/
molecular_properties/zinc/graphs/lipo/
molecular_properties/zinc/graphs/vp/
molecular_properties/zinc/graphs/bp/
molecular_properties/zinc/graphs/logp/
